# Ensemble Methods Tutorial for Hockey Prediction

This tutorial explains how to combine multiple models for better predictions.

## What You'll Learn

1. **Why Ensembles Work** - Wisdom of crowds
2. **Simple Averaging** - Combine equal-weight predictions
3. **Weighted Averaging** - Give better models more weight
4. **Stacking** - Use a meta-model to combine predictions
5. **Blending** - Practical ensemble strategy
6. **Practical Usage** - Building your ensemble

---

## 1. Why Ensembles Work

Different models make **different errors**. By combining them,
errors can cancel out!

### The Math

If we have 3 uncorrelated models with equal variance $\sigma^2$:

$$\text{Variance of average} = \frac{\sigma^2}{3}$$

**Error reduces as we add diverse models!**

### Key Insight
Ensembles work best when models are:
- **Different** (linear + tree + neural network)
- **Accurate** (each model should be decent on its own)
- **Uncorrelated** in their errors

### Common Ensemble Types

| Method | Description | When to Use |
|--------|-------------|-------------|
| Averaging | Mean of predictions | Quick improvement |
| Weighted | Weighted mean | Models have different skill |
| Stacking | Meta-model learns weights | Maximum accuracy |
| Blending | Like stacking, simpler | Practical compromise |

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Check for XGBoost
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    print("XGBoost not available, using GradientBoosting instead")

import warnings
warnings.filterwarnings('ignore')

print("Tutorial ready!")

In [ ]:
# Create sample hockey data
np.random.seed(42)
n = 600

data = pd.DataFrame({
    'elo_diff': np.random.normal(0, 100, n),
    'home_win_pct': np.random.uniform(0.35, 0.65, n),
    'away_win_pct': np.random.uniform(0.35, 0.65, n),
    'home_goals_avg': np.random.uniform(2.5, 3.5, n),
    'away_goals_against': np.random.uniform(2.5, 3.5, n),
    'home_pp_pct': np.random.uniform(0.15, 0.25, n),
    'rest_days': np.random.choice([1, 2, 3, 4], n),
})

# Target with both linear and non-linear components
# (so different models capture different aspects)
data['home_goals'] = (
    2.5 +
    # Linear component (Ridge will capture this)
    0.005 * data['elo_diff'] +
    0.6 * data['home_win_pct'] +
    # Non-linear component (Trees will capture this)
    np.where(data['elo_diff'] > 50, 0.4, 0) +
    np.where(data['rest_days'] >= 3, 0.3, 0) +
    # Interaction (complex models will capture this)
    0.3 * data['home_goals_avg'] * data['home_pp_pct'] * 4 +
    np.random.normal(0, 0.6, n)
).clip(0, 8).round().astype(int)

print(f"Data shape: {data.shape}")
data.head()

In [ ]:
# Split data: train / validation / test
feature_cols = [c for c in data.columns if c != 'home_goals']
X = data[feature_cols]
y = data['home_goals']

# 60% train, 20% validation (for ensemble weights), 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)}")

## 2. Training Base Models

Let's train diverse models that will form our ensemble.

In [ ]:
# Scale data for linear models and neural networks
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define base models
models = {
    'ridge': Ridge(alpha=1.0),
    'elastic': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'rf': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'mlp': MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=500, random_state=42),
}

if XGB_AVAILABLE:
    models['xgb'] = xgb.XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, 
                                     random_state=42, verbosity=0)
else:
    models['gb'] = GradientBoostingRegressor(n_estimators=100, max_depth=4, 
                                             learning_rate=0.1, random_state=42)

print(f"Training {len(models)} base models...")

In [ ]:
# Train all models and get predictions
val_predictions = {}
test_predictions = {}
model_scores = []

for name, model in models.items():
    # Use scaled data for linear/MLP, raw for trees
    if name in ['ridge', 'elastic', 'mlp']:
        model.fit(X_train_scaled, y_train)
        val_pred = model.predict(X_val_scaled)
        test_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        val_pred = model.predict(X_val)
        test_pred = model.predict(X_test)
    
    val_predictions[name] = val_pred
    test_predictions[name] = test_pred
    
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    model_scores.append({'model': name, 'val_rmse': rmse})
    print(f"  {name}: RMSE = {rmse:.4f}")

scores_df = pd.DataFrame(model_scores).sort_values('val_rmse')
print("\nModel Rankings:")
print(scores_df.to_string(index=False))

## 3. Simple Averaging

The simplest ensemble: just average all predictions!

$$\hat{y}_{ensemble} = \frac{1}{K} \sum_{k=1}^{K} \hat{y}_k$$

In [ ]:
# Simple average of all models
val_avg = np.mean(list(val_predictions.values()), axis=0)
test_avg = np.mean(list(test_predictions.values()), axis=0)

avg_rmse_val = np.sqrt(mean_squared_error(y_val, val_avg))
avg_rmse_test = np.sqrt(mean_squared_error(y_test, test_avg))

print(f"Simple Average Ensemble:")
print(f"  Validation RMSE: {avg_rmse_val:.4f}")
print(f"  Test RMSE:       {avg_rmse_test:.4f}")

# Compare to best single model
best_single = scores_df.iloc[0]
improvement = (best_single['val_rmse'] - avg_rmse_val) / best_single['val_rmse'] * 100
print(f"\nBest single model ({best_single['model']}): {best_single['val_rmse']:.4f}")
print(f"Improvement: {improvement:.1f}%")

## 4. Weighted Averaging

Give better models more influence:

$$\hat{y}_{ensemble} = \sum_{k=1}^{K} w_k \hat{y}_k, \quad \sum w_k = 1$$

### Methods to find weights:
1. **Inverse error**: $w_k \propto \frac{1}{RMSE_k}$
2. **Rank-based**: Better models get higher rank weights
3. **Optimization**: Find weights that minimize validation error

In [ ]:
# Method 1: Inverse RMSE weights
inverse_weights = {}
for name, pred in val_predictions.items():
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    inverse_weights[name] = 1 / rmse

# Normalize to sum to 1
total = sum(inverse_weights.values())
inverse_weights = {k: v/total for k, v in inverse_weights.items()}

print("Inverse RMSE Weights:")
for name, weight in sorted(inverse_weights.items(), key=lambda x: -x[1]):
    print(f"  {name}: {weight:.3f}")

In [ ]:
# Apply weighted average
val_weighted = np.zeros(len(y_val))
test_weighted = np.zeros(len(y_test))

for name, weight in inverse_weights.items():
    val_weighted += weight * val_predictions[name]
    test_weighted += weight * test_predictions[name]

weighted_rmse_val = np.sqrt(mean_squared_error(y_val, val_weighted))
weighted_rmse_test = np.sqrt(mean_squared_error(y_test, test_weighted))

print(f"Weighted Average Ensemble:")
print(f"  Validation RMSE: {weighted_rmse_val:.4f}")
print(f"  Test RMSE:       {weighted_rmse_test:.4f}")
print(f"\nCompared to simple average: {avg_rmse_val:.4f}")

In [ ]:
# Method 2: Optimized weights using scipy
try:
    from scipy.optimize import minimize
    
    # Stack predictions into matrix
    val_matrix = np.column_stack(list(val_predictions.values()))
    test_matrix = np.column_stack(list(test_predictions.values()))
    
    def objective(weights):
        pred = val_matrix @ weights
        return mean_squared_error(y_val, pred)
    
    # Constraints: weights sum to 1, all non-negative
    n_models = len(models)
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = [(0, 1) for _ in range(n_models)]
    
    # Initial guess: equal weights
    initial = np.ones(n_models) / n_models
    
    result = minimize(objective, initial, method='SLSQP', 
                      bounds=bounds, constraints=constraints)
    
    optimal_weights = result.x
    model_names = list(val_predictions.keys())
    
    print("Optimized Weights:")
    for name, weight in zip(model_names, optimal_weights):
        print(f"  {name}: {weight:.3f}")
    
    # Apply optimal weights
    val_optimal = val_matrix @ optimal_weights
    test_optimal = test_matrix @ optimal_weights
    
    optimal_rmse_val = np.sqrt(mean_squared_error(y_val, val_optimal))
    optimal_rmse_test = np.sqrt(mean_squared_error(y_test, test_optimal))
    
    print(f"\nOptimal Ensemble:")
    print(f"  Validation RMSE: {optimal_rmse_val:.4f}")
    print(f"  Test RMSE:       {optimal_rmse_test:.4f}")
    
except ImportError:
    print("scipy not available for optimization")

## 5. Stacking (Meta-Learning)

Use a **meta-model** to learn how to combine base model predictions.

### How It Works:
1. Train base models on training data
2. Get predictions from base models (using cross-validation!)
3. Train a meta-model using predictions as features
4. Final prediction = meta-model(base predictions)

### Why Cross-Validation?
If we use training predictions, base models have "seen" the data.
Their predictions are too good → meta-model overfits.

Using **out-of-fold predictions** gives honest estimates.

In [ ]:
# Stacking with sklearn
from sklearn.ensemble import StackingRegressor

# Define base estimators
base_estimators = [
    ('ridge', Ridge(alpha=1.0)),
    ('elastic', ElasticNet(alpha=0.1, l1_ratio=0.5)),
    ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
]

if XGB_AVAILABLE:
    base_estimators.append(
        ('xgb', xgb.XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, 
                                  random_state=42, verbosity=0))
    )
else:
    base_estimators.append(
        ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=4, 
                                          learning_rate=0.1, random_state=42))
    )

# Meta-model (simple Ridge works well)
stacking_model = StackingRegressor(
    estimators=base_estimators,
    final_estimator=Ridge(alpha=0.1),
    cv=5,  # 5-fold CV for out-of-fold predictions
    n_jobs=-1
)

print("Training stacking ensemble (this may take a moment)...")
stacking_model.fit(X_train_scaled, y_train)
print("Done!")

In [ ]:
# Evaluate stacking ensemble
stack_val_pred = stacking_model.predict(X_val_scaled)
stack_test_pred = stacking_model.predict(X_test_scaled)

stack_rmse_val = np.sqrt(mean_squared_error(y_val, stack_val_pred))
stack_rmse_test = np.sqrt(mean_squared_error(y_test, stack_test_pred))

print(f"Stacking Ensemble:")
print(f"  Validation RMSE: {stack_rmse_val:.4f}")
print(f"  Test RMSE:       {stack_rmse_test:.4f}")

In [ ]:
# What weights did the meta-model learn?
meta_model = stacking_model.final_estimator_
print("Meta-model coefficients (learned weights):")
for (name, _), coef in zip(base_estimators, meta_model.coef_):
    print(f"  {name}: {coef:.3f}")
print(f"  intercept: {meta_model.intercept_:.3f}")

## 6. Comparison Summary

In [ ]:
# Compare all methods
comparison = [
    {'Method': 'Best Single Model', 'Val RMSE': best_single['val_rmse'], 
     'Test RMSE': np.sqrt(mean_squared_error(y_test, test_predictions[best_single['model']]))},
    {'Method': 'Simple Average', 'Val RMSE': avg_rmse_val, 'Test RMSE': avg_rmse_test},
    {'Method': 'Weighted (Inverse RMSE)', 'Val RMSE': weighted_rmse_val, 'Test RMSE': weighted_rmse_test},
]

try:
    comparison.append({'Method': 'Weighted (Optimized)', 'Val RMSE': optimal_rmse_val, 'Test RMSE': optimal_rmse_test})
except:
    pass

comparison.append({'Method': 'Stacking', 'Val RMSE': stack_rmse_val, 'Test RMSE': stack_rmse_test})

comparison_df = pd.DataFrame(comparison)
print("Ensemble Method Comparison:")
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Val RMSE'], width, label='Validation', color='steelblue')
bars2 = ax.bar(x + width/2, comparison_df['Test RMSE'], width, label='Test', color='coral')

ax.set_ylabel('RMSE')
ax.set_title('Ensemble Method Comparison')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Method'], rotation=20, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

## 7. Practical Tips for Hockey Ensembles

### Recommended Ensemble

For hockey prediction, a simple weighted ensemble often works best:

```python
# Good starter ensemble
models = {
    'elo': EloModel(),           # Captures team strength dynamics
    'xgb': XGBRegressor(),        # Captures complex patterns
    'rf': RandomForestRegressor(), # Robust tree predictions
    'ridge': Ridge(),              # Captures linear relationships
}
```

### Do's and Don'ts

✅ **Do:**
- Use diverse model types (linear + trees + neural)
- Validate ensemble weights on held-out data
- Start with simple averaging
- Include domain-specific models (ELO for sports)

❌ **Don't:**
- Overfit ensemble weights on validation data
- Use too many similar models (3 different RF = just one RF)
- Forget to retrain on full data for production
- Make ensembles too complex (diminishing returns)

In [ ]:
# Summary
print("="*50)
print(" ENSEMBLE RECOMMENDATIONS FOR HOCKEY")
print("="*50)
print("""
1. QUICK WIN: Simple average of 3-4 diverse models
   - Usually gives 5-15% improvement

2. BETTER: Inverse RMSE weighted average
   - Easy to implement, no extra training

3. BEST: Stacking with Ridge meta-model
   - Learns optimal combination automatically
   - Use 5-fold CV for out-of-fold predictions

RECOMMENDED BASE MODELS:
- EloModel (captures rating dynamics)
- XGBRegressor (best for complex patterns)
- RandomForest (robust, less tuning needed)
- Ridge (captures linear relationships)

TYPICAL WEIGHTS (goals prediction):
- XGBoost: 35-45%
- Random Forest: 25-35%
- ELO: 15-25%
- Linear: 10-15%
""")
print("Tutorial complete!")